In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, Model, applications
from ultralytics import YOLO
from typing import Tuple, List, Dict, Optional

# --------------------------
# 1. ENHANCED DATA LOADER
# --------------------------
class KITTILoader:
    def __init__(self, base_path: str):
        self.base_path = base_path
        self.class_map = {"Car": 0, "Pedestrian": 1, "Cyclist": 2}
        self._validate_paths()

    def _validate_paths(self):
        """Verify critical dataset folders exist"""
        required_folders = [
            "data_object_image_2/training/image_2",
            "data_object_image_3/training/image_3",
            "data_object_label_2/training/label_2"
        ]
        for folder in required_folders:
            if not os.path.exists(os.path.join(self.base_path, folder)):
                raise FileNotFoundError(f"❌ Missing required folder: {folder}")

    def load_stereo_pair(self, split: str, idx: int) -> Tuple[np.ndarray, np.ndarray]:
        """Loads and preprocesses stereo image pairs"""
        img_paths = [
            os.path.join(self.base_path, f"data_object_image_2/{split}/image_2/{idx:06d}.png"),
            os.path.join(self.base_path, f"data_object_image_3/{split}/image_3/{idx:06d}.png")
        ]
        
        imgs = []
        for path in img_paths:
            if not os.path.exists(path):
                raise FileNotFoundError(f"❌ Image not found: {path}")
            
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            img = cv2.resize(img, (416, 416))
            imgs.append(img)
            
        return tuple(imgs)

    def load_labels(self, split: str, idx: int) -> List[Dict]:
        """Loads and converts labels to COCO format with enhanced validation"""
        label_path = os.path.join(self.base_path, f"data_object_label_2/{split}/label_2/{idx:06d}.txt")
        
        if not os.path.exists(label_path):
            return []  # Return empty list if no labels exist

        labels = []
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 15:  # Minimum fields in KITTI label
                    continue
                    
                if parts[0] in self.class_map:
                    try:
                        x1, y1, x2, y2 = map(float, parts[4:8])
                        labels.append({
                            "class": self.class_map[parts[0]],
                            "bbox": [x1, y1, x2 - x1, y2 - y1],  # COCO format
                            "truncated": float(parts[1]),
                            "occluded": int(parts[2])
                        })
                    except (ValueError, IndexError) as e:
                        print(f"⚠️ Error parsing label {label_path}: {e}")
                        
        return labels

    def load_calibration(self, split: str, idx: int) -> Optional[Dict]:
        """Loads calibration data with error handling"""
        calib_path = os.path.join(self.base_path, f"data_object_calib/{split}/calib/{idx:06d}.txt")
        
        if not os.path.exists(calib_path):
            print(f"⚠️ Calibration file not found: {calib_path}")
            return None

        calib = {}
        with open(calib_path, 'r') as f:
            for line in f:
                try:
                    key, *values = line.strip().split(' ', 1)
                    calib[key] = np.array([float(x) for x in values[0].split()]).reshape(3, 4)
                except (ValueError, IndexError) as e:
                    print(f"⚠️ Error parsing calibration {calib_path}: {e}")
                    
        return calib

# --------------------------
# 2. IMPROVED SIAMESE NETWORK
# --------------------------
class StereoVerifier:
    def __init__(self, input_shape=(416, 416, 3)):
        self.input_shape = input_shape
        self.model = self._build_enhanced_model()
        
    def _build_enhanced_model(self) -> Model:
        """Enhanced Siamese network with feature normalization"""
        base_cnn = applications.ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=self.input_shape
        )
        
        # Freeze early layers
        for layer in base_cnn.layers[:100]:
            layer.trainable = False
            
        # Feature normalization
        global_average = layers.GlobalAveragePooling2D()
        normalization = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
        
        # Twin networks
        input_left = layers.Input(self.input_shape)
        input_right = layers.Input(self.input_shape)
        
        features_left = normalization(global_average(base_cnn(input_left)))
        features_right = normalization(global_average(base_cnn(input_right))))
        
        # Distance metric
        distance = layers.Lambda(
            lambda x: tf.reduce_sum(tf.square(x[0] - x[1]), axis=1, keepdims=True)
        )([features_left, features_right])
        
        # Verification head
        output = layers.Dense(1, activation='sigmoid')(distance)
        
        return Model(inputs=[input_left, input_right], outputs=output)

    def train(self, loader: KITTILoader, epochs=10, batch_size=32):
        """Enhanced training with validation split"""
        # Generate balanced pairs
        pairs, labels = [], []
        sample_count = min(200, len(os.listdir(os.path.join(loader.base_path, "data_object_image_2/training/image_2")))
        
        for i in range(sample_count):
            try:
                left, right = loader.load_stereo_pair("training", i)
                pairs.append([left, right])
                labels.append(1)  # Positive pair
                
                # Negative pair (different stereo pair)
                if i > 0:
                    _, neg_right = loader.load_stereo_pair("training", i-1)
                    pairs.append([left, neg_right])
                    labels.append(0)
            except Exception as e:
                print(f"⚠️ Error loading sample {i}: {e}")
                continue
                
        # Convert to numpy arrays
        left_imgs = np.array([p[0] for p in pairs])
        right_imgs = np.array([p[1] for p in pairs])
        labels = np.array(labels)
        
        # Add early stopping
        callback = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
        
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(0.0001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        history = self.model.fit(
            [left_imgs, right_imgs],
            labels,
            validation_split=0.2,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[callback]
        )
        
        return history

# --------------------------
# 3. ADVANCED STEREO GAN
# --------------------------
class StereoGAN:
    def __init__(self, latent_dim=100, img_shape=(416, 416, 3)):
        self.latent_dim = latent_dim
        self.img_shape = img_shape
        self.generator = self._build_generator()
        self.discriminator = self._build_discriminator()
        self.gan = self._build_combined()
        
    def _build_generator(self) -> Model:
        """Generator with skip connections"""
        noise = layers.Input(shape=(self.latent_dim,))
        
        # Shared encoder
        x = layers.Dense(512)(noise)
        x = layers.LeakyReLU(0.2)(x)
        x = layers.BatchNormalization()(x)
        
        # Left image decoder
        left = layers.Dense(256)(x)
        left = layers.LeakyReLU(0.2)(left)
        left = layers.Dense(np.prod(self.img_shape), activation='tanh')(left)
        left = layers.Reshape(self.img_shape)(left)
        
        # Right image decoder with disparity
        right = layers.Dense(256)(x)
        right = layers.LeakyReLU(0.2)(right)
        right = layers.Dense(np.prod(self.img_shape), activation='tanh')(right)
        right = layers.Reshape(self.img_shape)(right)
        
        return Model(noise, [left, right])

    def _build_discriminator(self) -> Model:
        """Discriminator with dual input for stereo pairs"""
        left_input = layers.Input(self.img_shape)
        right_input = layers.Input(self.img_shape)
        
        # Shared feature extractor
        def create_feature_extractor():
            model = tf.keras.Sequential([
                layers.Conv2D(64, (4,4), strides=2, padding='same'),
                layers.LeakyReLU(0.2),
                layers.Conv2D(128, (4,4), strides=2, padding='same'),
                layers.LeakyReLU(0.2),
                layers.GlobalMaxPooling2D()
            ])
            return model
            
        left_features = create_feature_extractor()(left_input)
        right_features = create_feature_extractor()(right_input)
        
        # Combine features
        merged = layers.Concatenate()([left_features, right_features])
        validity = layers.Dense(1, activation='sigmoid')(merged)
        
        return Model([left_input, right_input], validity)

    def _build_combined(self) -> Model:
        """Combined GAN model"""
        self.discriminator.compile(
            optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
            loss='binary_crossentropy'
        )
        self.discriminator.trainable = False
        
        noise = layers.Input(shape=(self.latent_dim,))
        img_left, img_right = self.generator(noise)
        valid = self.discriminator([img_left, img_right])
        
        return Model(noise, valid)

    def train(self, real_left: np.ndarray, real_right: np.ndarray, epochs=100, batch_size=32):
        """Enhanced GAN training with gradient penalty"""
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            # ---------------------
            # Train Discriminator
            # ---------------------
            idx = np.random.randint(0, real_left.shape[0], batch_size)
            real_imgs_left = real_left[idx]
            real_imgs_right = real_right[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            fake_left, fake_right = self.generator.predict(noise, verbose=0)
            
            # Train on real and fake images
            d_loss_real = self.discriminator.train_on_batch(
                [real_imgs_left, real_imgs_right], valid
            )
            d_loss_fake = self.discriminator.train_on_batch(
                [fake_left, fake_right], fake
            )
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # ---------------------
            # Train Generator
            # ---------------------
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.gan.train_on_batch(noise, valid)
            
            # Print progress
            if epoch % 10 == 0:
                print(f"Epoch {epoch} [D loss: {d_loss[0]} | G loss: {g_loss}]")

# --------------------------
# 4. ENHANCED YOLO DETECTOR
# --------------------------
class KITTIDetector:
    def __init__(self, model_type='yolov8n.pt'):
        self.model = YOLO(model_type)
        self.class_map = {0: 'Car', 1: 'Pedestrian', 2: 'Cyclist'}
        
    def train(self, data_yaml='kitti.yaml', epochs=50, imgsz=416):
        """Enhanced training with validation"""
        results = self.model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=imgsz,
            batch=16,
            patience=10,  # Early stopping
            augment=True,  # Mosaic augmentation
            cache=True    # Cache images for faster training
        )
        return results
    
    def detect_stereo(self, left_img: np.ndarray, right_img: np.ndarray) -> Tuple[np.ndarray, list]:
        """Enhanced stereo detection with NMS"""
        # Run detection on both images
        left_results = self.model(left_img, verbose=False)[0]
        right_results = self.model(right_img, verbose=False)[0]
        
        # Process detections
        def process_detections(results):
            boxes = []
            for box in results.boxes:
                if box.conf > 0.5:  # Confidence threshold
                    x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                    class_id = int(box.cls)
                    boxes.append({
                        'bbox': [x1, y1, x2, y2],
                        'class': self.class_map.get(class_id, 'Unknown'),
                        'confidence': float(box.conf)
                    })
            return boxes
            
        left_detections = process_detections(left_results)
        right_detections = process_detections(right_results)
        
        # Visualize on left image
        output_img = left_img.copy()
        for det in left_detections + right_detections:
            x1, y1, x2, y2 = det['bbox']
            cv2.rectangle(output_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                output_img,
                f"{det['class']} {det['confidence']:.2f}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1
            )
            
        return output_img, left_detections + right_detections

# --------------------------
# MAIN EXECUTION
# --------------------------
def main():
    # Initialize components
    loader = KITTILoader("D:/kitti_dataset")
    verifier = StereoVerifier()
    detector = KITTIDetector()
    
    try:
        # 1. Load sample data
        print("🔍 Loading sample data...")
        left_img, right_img = loader.load_stereo_pair("training", 0)
        labels = loader.load_labels("training", 0)
        print(f"✅ Loaded sample with {len(labels)} objects")
        
        # 2. Train Siamese Network
        print("\n🎯 Training Siamese Network...")
        verifier.train(loader, epochs=15)
        
        # 3. Train Object Detector
        print("\n🎯 Training YOLOv8 Detector...")
        detector.train(epochs=50)
        
        # 4. Run Detection
        print("\n🔍 Running Stereo Detection...")
        test_left, test_right = loader.load_stereo_pair("testing", 0)
        result_img, detections = detector.detect_stereo(test_left, test_right)
        
        # Display results
        cv2.imshow("Stereo Detections", cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR))
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        print(f"\n🎉 Detected {len(detections)} objects in stereo pair")
        
    except Exception as e:
        print(f"❌ Error in pipeline: {str(e)}")

if __name__ == "__main__":
    main()